# **Penting**
- Pastikan Anda melakukan Run All sebelum mengirimkan submission untuk memastikan seluruh cell berjalan dengan baik.
- Hapus simbol pagar (#) jika Anda menerapkan kriteria tambahan
- Biarkan simbol pagar (#) jika Anda tidak menerapkan kriteria tambahan

# **1. Import Library**
Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
import joblib
from sklearn.model_selection import RandomizedSearchCV

# **2. Memuat Dataset dari Hasil Clustering**
Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [ ]:
# Gunakan dataset hasil clustering yang memiliki fitur Target
# Silakan gunakan dataset data_clustering jika tidak menerapkan Interpretasi Hasil Clustering [Advanced]
# Silakan gunakan dataset data_clustering_inverse jika menerapkan Interpretasi Hasil Clustering [Advanced]
file_id = '1kGoXu3rthgPA8Nh-D8yE7wM7S3Z0tNHb'
download_url = f'https://drive.google.com/uc?id={file_id}'

df = pd.read_csv(download_url)

In [ ]:
# Tampilkan 5 baris pertama dengan function head.

In [ ]:
df.head()

,TransactionAmount,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance,LocationCoords,Month,TransactionType,Channel,CustomerOccupation,AgeGroup,AmountGroup,Target
0,14.09,70.0,81.0,1.0,5112.21,3915.340577,4,0,0,0,2,1,0
1,376.24,68.0,141.0,1.0,13758.91,2283.538695,6,0,0,0,2,1,1
2,126.29,19.0,56.0,1.0,1122.35,3431.992748,7,0,1,2,0,1,0
3,184.50,26.0,25.0,1.0,8569.06,681.143085,5,0,1,2,0,1,2
4,92.15,18.0,172.0,1.0,781.68,2135.713474,4,0,0,2,0,1,1


In [ ]:
df.isna().sum()

,0
TransactionAmount,0
CustomerAge,0
TransactionDuration,0
LoginAttempts,0
AccountBalance,0
LocationCoords,0
Month,0
TransactionType,0
Channel,0
CustomerOccupation,0


# **3. Data Splitting**
Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [ ]:
# Menggunakan train_test_split() untuk melakukan pembagian dataset.
X = df.drop(columns=['Target', 'LocationCoords'])
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Penjelasan:
- Melakukan data splitting untuk melakukan pembagian dataset
- Disini untuk **label** saya menggunakan hasil dari cluster nya, lalu untuk X, tentu saja saya menghapus kolom yang menjadi label, dan juga disini saya menghapus kolom 'LocationCoords' ternyata kolom ini adalah data leakage (saya tahu karna sudah melakukan beberapa pengecekan sebelumnya dari yg awalnya 'Location' sampai yang 'LocationCoords' sama-sama data leakage)
- jadi daripada saya mendapatkan skor tinggi tapi dengan model yang tidak bagus, saya memilih untuk menghapusnya

# **4. Membangun Model Klasifikasi**
Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Menggunakan algoritma klasifikasi yaitu Decision Tree.
2. Latih model menggunakan data yang sudah dipisah.

In [ ]:
# Buatlah model klasifikasi menggunakan Decision Tree

dt = DecisionTreeClassifier().fit(X_train, y_train)

In [ ]:
# Menyimpan Model
joblib.dump(dt, 'decision_tree_model.h5')

['decision_tree_model.h5']

# **5. Memenuhi Kriteria Skilled dan Advanced dalam Membangun Model Klasifikasi**



**Biarkan kosong jika tidak menerapkan kriteria skilled atau advanced**

In [ ]:
# Melatih model menggunakan algoritma klasifikasi selain Decision Tree.

knn = KNeighborsClassifier().fit(X_train, y_train)
rf = RandomForestClassifier().fit(X_train, y_train)
svm = SVC().fit(X_train, y_train)
nb = GaussianNB().fit(X_train, y_train)

Penjelasan:
- Jadi disini saya menggunakan 4 algoritma klasifikasi selain Decision Tree yaitu KNN, Random Forest, SVM, dan Gaussian NB

In [ ]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada seluruh algoritma yang sudah dibuat.

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)

    results = {
        'Confusion Matrix': cm,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted'),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1-Score': f1_score(y_test, y_pred, average='weighted')
    }
    return results

# Mengevaluasi setiap model dan mengumpulkan hasilnya
results = {
    'K-Nearest Neighbors (KNN)': evaluate_model(knn, X_test, y_test),
    'Decision Tree (DT)': evaluate_model(dt, X_test, y_test),
    'Random Forest (RF)': evaluate_model(rf, X_test, y_test),
    'Support Vector Machine (SVM)': evaluate_model(svm, X_test, y_test),
    'Naive Bayes (NB)': evaluate_model(nb, X_test, y_test)
}

# Buat DataFrame untuk meringkas hasil
summary_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Isi DataFrame dengan hasil
rows = []
for model_name, metrics in results.items():
    rows.append({
        'Model': model_name,
        'Accuracy': metrics['Accuracy'],
        'Precision': metrics['Precision'],
        'Recall': metrics['Recall'],
        'F1-Score': metrics['F1-Score']
    })

# Konversi daftar kamus ke DataFrame
summary_df = pd.DataFrame(rows)

# Tampilkan DataFrame
print(summary_df)

                          Model  Accuracy  Precision    Recall  F1-Score
0     K-Nearest Neighbors (KNN)  0.324455   0.339960  0.324455  0.325327
1            Decision Tree (DT)  0.363196   0.367587  0.363196  0.364908
2            Random Forest (RF)  0.365617   0.360294  0.365617  0.360989
3  Support Vector Machine (SVM)  0.418886   0.258195  0.418886  0.286618
4              Naive Bayes (NB)  0.389831   0.374575  0.389831  0.368861


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Insight:
- Pada hasil evaluasi diatas, semua model memiliki nilai evaluasi yang kurang bagus, dengan nilai evaluasi tertinggi hanya 0.37 yang dimiliki oleh model Naive Bayes, sedangkan yang paling rendah adalah 0.29 yang dimiliki oleh model SVM

In [ ]:
# Menyimpan Model Selain Decision Tree
# Model ini bisa lebih dari satu

joblib.dump(knn, 'explore_KNN_classification.h5')
joblib.dump(rf, 'explore_RF_classification.h5')
joblib.dump(svm, 'explore_SVM_classification.h5')
joblib.dump(nb, 'explore_NB_classification.h5')

['explore_NB_classification.h5']

**Hyperparameter Tuning**

Hyperparameter Tuning Model

Pilih salah satu algoritma yang ingin Anda tuning

In [ ]:
pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 4.4 MB/s eta 0:00:00


Penjelasan:
- Melakukan install scikit optimize agar model Bayesian Optimization bisa dipanggil

In [ ]:
from skopt import BayesSearchCV

# Definisikan ruang pencarian untuk Bayesian Optimization
param_space = {
    'n_estimators': (100, 500),
    'max_depth': (10, 50),
    'min_samples_split': (2, 10),
    'criterion': ['gini', 'entropy']
}

# Inisialisasi BayesSearchCV
bayes_search = BayesSearchCV(estimator=rf, search_spaces=param_space, n_iter=32, cv=3, n_jobs=-1, verbose=2, random_state=42)
bayes_search.fit(X_train, y_train)

# Output hasil terbaik
print(f"Best parameters (Bayesian Optimization): {bayes_search.best_params_}")
best_rf_bayes = bayes_search.best_estimator_

# Evaluasi performa model pada test set
bayes_search_score = best_rf_bayes.score(X_test, y_test)
print(f"Accuracy after Bayesian Optimization: {bayes_search_score:.2f}")

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


Penjelasan:
- Ini sebenarnya kalau saya lihat dari tugas submission, yang harusnya di tuning adalah model yang memiliki nilai evalasi terendah, dan pada kasus saya, model tersebut adalah SVM.
- Tapi disini saya melakukan tuning pada algoritma Random Forest, yang mana memiliki akurasi paling tinggi, dikarenakan saya sebelumnya sudah mencoba berkali-kali untuk melakukan tuning pada algoritma SVM, tetapi proses executingnya sangatlah lama bahkan sudah sampai 2 jam masih belum selesai tuningnya walau sudah memakai berbagai macam model tuning, jadi saya putuskan untuk menggunakan model Random Forest saja karena memang model ini model yang memiliki evaluasi paling bagus, tetapi model ini salah satu model yang bisa untuk saya lakukan proses tuning, jadi daripada proses pada bagian ini kosong disebabkan oleh proses tuning pada model SVM yang sangat lama, jadi saya putuskan untuk mengisinya dengan model yang memiliki evaluasi paling tinggi yaitu Random Forest
- Disini saya menggunakan algoritma Bayesian Optimization karena algoritma ini adalah yang paling optimal untuk menaikkan akurasi dari model Random Forest saya, bisa dilihat setelah proses tuning, nilai akurasi yang dihasilkan naik cukup tinggi dari 0.36 ke 0.41

**Hasil Evaluasi dari Algoritma yang Sudah Dituning**

In [ ]:
y_pred = best_rf_bayes.predict(X_test)

# Hitung metrik evaluasi
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Tampilkan hasilnya
print("Hasil Evaluasi Model (Setelah Tuning):")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-Score : {f1:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Insight:
- Skor akurasi yang dimiliki model sekarang menjadi naik dari 0.36 ke 0.41, precision dari 0.36 ke 0.37, recall dari 0.36 ke 0.41, dan f1-scorenya naik dari 0.36 ke 0.37

In [ ]:
# Menyimpan Model hasil tuning
joblib.dump(best_rf_bayes, 'tuning_classification.h5')